In [1]:
from pointCluster import PointCluster
from csvAnalysis import CSVAnalysis
from placesGeometry import PlacesGeometry
from poi import Poi
from toGeoPackage import ToGeoPackage

print("Optimal Placement of EV Charging Stations in Georgia State")

# Declare data file paths
poiCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\georgia_pois.csv')
vehicleCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\vehicle_hotspot_pois.csv')
placesCSV = CSVAnalysis('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\places_coords.csv')
county_info = placesCSV.read_places('C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\county_pop_dens.csv', 'C:\\Users\\vzhang\\workplace\\EVMapping\\placement_algorithm\\county_inc.csv')

# Read data files and extract POIs
poi_lat_lons = poiCSV.create_lat_lons()
raw_lat_lons = vehicleCSV.raw_points()
vehicle_lat_lons = vehicleCSV.create_lat_lons()
vehicle_pois = vehicleCSV.read_points()

# Filter parking locations to have fewer points
vehicle_lat_lons = PointCluster.filter_close_points(vehicle_lat_lons)

# Cluster the combination of popular POIs and vehicle POIs
cluster = PointCluster()
all_coords = PointCluster.combine_clusters(poi_lat_lons, vehicle_lat_lons)
cluster.coords = all_coords
cluster.point_cluster()

# Adjust pois to vehicle hotspots
adjusted_popular = cluster.adjust_points(vehicle_lat_lons, vehicle_pois)

# Determine hotspot capacities based on filtered points
vehicle_pois = PointCluster.assign_weights(adjusted_popular, vehicle_pois)

# Determine which counties do not have EV points within them
remaining_counties = PointCluster.cluster_counties(PlacesGeometry.find_remaining_counties(poi_lat_lons, county_info), poi_lat_lons)

# Add chargers to promote diversity and accessibility
diversity = PointCluster.assign_weights(PointCluster.adjust_chargers(raw_counties, vehicle_lat_lons), vehicle_pois)
remaining = PointCluster.assign_weights(PointCluster.adjust_chargers(remaining_counties, vehicle_lat_lons), vehicle_pois)

# Second run of clustering based on counties
raw_counties = PointCluster.cluster_counties(county_info, poi_lat_lons)

finals = vehicle_pois + diversity + remaining
finals = Poi.correct_names(finals)

# Create new ToGeoPackage object to write file
toGeoPackage = ToGeoPackage()
toGeoPackage.extract_data(finals)
toGeoPackage.write_file()

Optimal Placement of EV Charging Stations in Georgia State
Reading coordinate file for places...
Reading population data...
Reading income data...
Reading CSV file for points...
Extracting latitude and longitude from Poi objects...
Reading CSV file for points...
Reading CSV file for points...
Extracting latitude and longitude from Poi objects...
Reading CSV file for points...
Identifying diversity counties...
Calculating additional chargers...
Starting point clustering for counties...
Filtering points...
Combining clusters...
Starting point clustering...
Calculating points per cluster...
Number of clusters found: 481
Adjusting points...
Assigning Weights...
Finding remaining counties...
|████████████████████████████████████████| 31416/31416 [100%] in 35:58.6 (14.55/ ▅▃▁ 345/31416 [1%] in 24s (~36:36, 14 ▂▄▆ 2153/31416 [7%] in 1:57 (~26:49,  ▇▅▃ 2759/31416 [9%] in 2:39 (~27:30,  ▆█▆ 4672/31416 [15%] in 4:39 (~26:35, ▄▂▂ 5034/31416 [16%] in 5:04 (~26:31, ▇▇▅ 5644/31416 [18%] in 5:46 (~26